In [1]:
import QUANTAXIS as QA
import pandas as pd
import numpy as np
from database import DB_CONN
from pymongo import ASCENDING, UpdateOne

In [2]:
start = '2015-01-01'
end = '2018-09-30'

index_code_list = ['000001', '000300', '399001', '399005', '399006']
_df = pd.read_excel('data/stock_basic.xlsx', header=0, dtype={'code':str})
stock_code_list = _df.code.tolist()

In [51]:
# 替换成stock code就好
df = QA.QA_fetch_index_day_adv(index_code_list, start, end).data
df_nonind = df.reset_index()
df_nonind.date = df_nonind.date.apply(lambda x : x.strftime('%Y-%m-%d'))

In [32]:
daily = DB_CONN['daily']
daily_hfq = DB_CONN['daily_hfq']
daily.create_index(
    [('code',ASCENDING), ('date',ASCENDING), ('index',ASCENDING)])
daily_hfq.create_index(
    [('code',ASCENDING), ('date',ASCENDING), ('index',ASCENDING)])

'code_1_date_1_index_1'

In [58]:
import time

In [60]:
tic = time.process_time()

update_requests = []
for i in df_nonind.index:
    doc = dict(df_nonind.loc[i, ['code', 'date', 'open', 'high', 'low', 'close','volume',
                      ]])
    doc.update({'index':True})
    update_requests.append(
        UpdateOne(
        {'code':doc['code'], 'date':doc['date']},
        {'$set':doc},
        upsert=True)
    )
if len(update_requests) > 0:
    update_result = daily.bulk_write(update_requests, ordered=False)
    print('保存日线数据,插入：%4d条, 更新：%4d条' %
      (update_result.upserted_count, update_result.modified_count),
      flush=True)

toc = time.process_time()
print('cost time : %.2fs' % (toc-tic))

保存日线数据,插入：4575条, 更新：   0条
cost time : 4.88s


In [46]:
# df_nonind = df_nonind.assign(index=True)

In [55]:
# df_nonind.loc[0]

In [56]:
# update_result = [dict(df_nonind.loc[i, ['code', 'date', 'open', 'high', 'low', 'close','volume',
#                       'index']]) for i in df_nonind.index]


In [57]:
# from pymongo import UpdateMany

In [1]:
import datetime, time
from pymongo import UpdateOne, ASCENDING, UpdateMany
from database import DB_CONN
from stock_util import get_trading_dates, get_all_codes
import tushare as ts
import numpy as np
import pandas as pd
import requests
import json
import datetime

In [5]:
code = '000004'
daily = DB_CONN['daily']

In [6]:
cursor = daily.find({'code':code, 'index':False})
df_res = pd.DataFrame([r for r in cursor])
df_res = df_res.drop(columns=['_id'])
df_res.head()

,au_factor,close,code,date,high,high_limit,index,is_st,is_trading,low,low_limit,open,pe,pre_close,volume
0,1.0,15.69,000004,2015-01-05,15.90,NaN,False,False,True,15.30,NaN,15.58,656.4854,NaN,28095.0
1,1.0,16.46,000004,2015-01-06,16.56,17.26,False,False,True,15.50,14.12,15.65,688.7029,15.69,33378.0
2,1.0,16.41,000004,2015-01-07,16.54,18.11,False,False,True,16.11,14.81,16.45,686.6109,16.46,20737.0
3,1.0,16.92,000004,2015-01-08,17.03,18.05,False,False,True,16.24,14.77,16.41,707.9498,16.41,36450.0
4,1.0,16.43,000004,2015-01-09,16.95,18.61,False,False,True,16.26,15.23,16.92,687.4477,16.92,27150.0


In [17]:
update_requests = []
update_requests.append(
    UpdateMany(
    {'code':code, 'index':False},
    {'$set':{'is_st':False}},
    upsert=True)
)

if len(update_requests)>0:
    update_result = daily.bulk_write(update_requests, ordered=False)
    print('填充字段， 字段名: is_st，数据集：%s，插入：%4d条，更新：%4d条' %
              ('daily', update_result.upserted_count, update_result.modified_count), flush=True)

填充字段， 字段名: is_st，数据集：daily，插入：   0条，更新： 741条


In [7]:
update_requests = []
df = pd.read_excel('data/stock_basic.xlsx', header=0, dtype={'code':str})
df = df.set_index('code')
today = str(datetime.date.today())

In [8]:
st_date_info = df.loc[code, '戴帽摘帽日期'].split(',')

st_interval_list = []
temp_st_end = None
for count, i in enumerate(st_date_info):
    key = i.split(':')[0]
    value = datetime.datetime.strptime(i.split(':')[1], '%Y-%m-%d')

    if (key == '去ST' or key == '去*ST') and count==0:
        temp_st_end = (value - datetime.timedelta(1)).strftime('%Y-%m-%d')

    elif (key == '去ST' or key == '去*ST') and count >0:
        temp_st_start = datetime.datetime.strptime(st_date_info[count-1].split(':')[1], 
                                   '%Y-%m-%d').strftime('%Y-%m-%d')
        st_interval_list.append([temp_st_start, temp_st_end])
        temp_st_end = (value - datetime.timedelta(1)).strftime('%Y-%m-%d')
    elif count == (len(st_date_info)-1):
        if temp_st_end is not None:
            st_interval_list.append([value.strftime('%Y-%m-%d'), temp_st_end])
        else:
            st_interval_list.append([value.strftime('%Y-%m-%d'), today])



In [9]:
st_interval_list

[['2011-06-08', '2011-06-07'],
 ['2009-04-21', '2010-05-30'],
 ['2006-04-21', '2007-05-24'],
 ['1999-04-27', '2001-03-05']]

In [26]:
for interval in st_interval_list:
    _start, _end = interval
    update_requests.append(
        UpdateMany(
        {'code':code, 'date':{'$gte':_start, '$lte':_end}, 'index':False},
        {'$set':{'is_st':True}}))

if len(update_requests)>0:
    update_result = daily.bulk_write(update_requests, ordered=False)
    print('填充字段， 字段名: is_st，数据集：%s，插入：%4d条，更新：%4d条' %
              ('daily', update_result.upserted_count, update_result.modified_count), flush=True)

#     print('第二阶段填充进度: %s/%s' %(i+1, total))

填充字段， 字段名: is_st，数据集：daily，插入：   0条，更新：   0条


In [36]:
codes = df.index.tolist()

In [50]:
df_temp = pd.Series()
for ind, code in enumerate(codes):
    if pd.isna(df.loc[code, '戴帽摘帽日期']):
        df_temp.loc[code] = '--'
        continue
        
    st_date_info = df.loc[code, '戴帽摘帽日期'].split(',')

    st_interval_list = []
    temp_st_end = None
    for count, i in enumerate(st_date_info):
        key = i.split(':')[0]
        value = datetime.datetime.strptime(i.split(':')[1], '%Y-%m-%d')

        if (key == '去ST' or key == '去*ST') and count==0:
            temp_st_end = (value - datetime.timedelta(1)).strftime('%Y-%m-%d')

        elif (key == '去ST' or key == '去*ST') and count >0:
            temp_st_start = datetime.datetime.strptime(st_date_info[count-1].split(':')[1], 
                                       '%Y-%m-%d').strftime('%Y-%m-%d')
            if temp_st_end is None:
                st_interval_list.append([temp_st_start, today])
            else:
                st_interval_list.append([temp_st_start, temp_st_end])
            temp_st_end = (value - datetime.timedelta(1)).strftime('%Y-%m-%d')
        elif count == (len(st_date_info)-1):
            if temp_st_end is not None:
                st_interval_list.append([value.strftime('%Y-%m-%d'), temp_st_end])
            else:
                st_interval_list.append([value.strftime('%Y-%m-%d'), today])
                
    df_temp.loc[code] = st_interval_list

In [51]:
df = df.assign(戴帽摘帽日期列表=df_temp)

In [53]:
df.head(10)

,证券代码,证券名称,首发上市日期,首发价格,戴帽摘帽日期,戴帽摘帽日期列表
code,,,,,,
000001,000001.SZ,平安银行,1991-04-03,40.0,NaN,--
000002,000002.SZ,万科A,1991-01-29,1.0,NaN,--
000004,000004.SZ,国农科技,1991-01-14,1.0,"去ST:2011-06-08,去*ST:2010-05-31,*ST:2009-04-21,...","[[2006-04-21, 2011-06-07], [1999-04-27, 2001-0..."
000005,000005.SZ,世纪星源,1990-12-10,10.0,"去ST:2008-06-25,去*ST:2004-05-18,*ST:2003-05-12,...","[[2003-04-16, 2008-06-24]]"
000006,000006.SZ,深振业A,1992-04-27,10.0,NaN,--
000007,000007.SZ,全新好,1992-04-13,1.0,"去ST:2012-05-04,ST:2007-04-23,去ST:2002-04-12,ST...","[[2007-04-23, 2012-05-03], [2001-04-19, 2002-0..."
000008,000008.SZ,神州高铁,1992-05-07,7.9,"去ST:2013-04-19,去*ST:2008-04-03,*ST:2007-03-30,...","[[2007-03-30, 2013-04-18], [2002-04-30, 2005-0..."
000009,000009.SZ,中国宝安,1991-06-25,3.0,"去ST:2000-04-14,ST:1999-04-30","[[1999-04-30, 2000-04-13]]"
000010,000010.SZ,美丽生态,1995-10-27,1.0,"去ST:2013-10-18,去*ST:2007-03-21,*ST:2006-05-09,...","[[2006-05-09, 2013-10-17], [2000-04-27, 2001-0..."


In [54]:
writer = pd.ExcelWriter('data/pandas_simple.xlsx', engine='xlsxwriter')

In [55]:
df.to_excel(writer)
writer.save()